## xcube Data Store Framework - Multi-Level Datasets

*Please checkout the general introduction to xcube data stores in the Jupyter Notebook [Getting Started](./1_getting_started.ipynb) before jumping into this notebook :)* 

This notebook explains how to generate spatial multi-level / multi-resolution datasets (image pyramids) using the `file` or `s3` data stores.

Multi-level datasets can be used with the xcube server (`xcube serve` CLI) to speed up visualisation of large data cubes.

In [1]:
from xcube.core.new import new_cube
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.store import new_data_writer

### Getting prepared

In [2]:
root = "testdata"  # Directory for test data

In [3]:
def new_test_cube():
    """Generate some test data"""
    return new_cube(width=3600, 
                    height=1800, 
                    x_res=0.1, 
                    variables=dict(A=1, B=2))

In [4]:
import shutil
shutil.rmtree(root, ignore_errors=True)  # Remove existing test data

Get a data store instance. We use the local filesystem here ("file"), you can also AWS S3 ("s3") if you have a writable bucket (= root).

In [5]:
data_store = new_data_store("file", root=root)  # Could also use "s3"

Get available data writers:

In [6]:
data_store.get_data_writer_ids()

('dataset:netcdf:file',
 'dataset:zarr:file',
 'dataset:levels:file',
 'mldataset:levels:file',
 'geodataframe:shapefile:file',
 'geodataframe:geojson:file')

Get the parameter schema writing datasets using the Zarr format (\*.zarr):

In [7]:
data_store.get_write_data_params_schema(writer_id='dataset:zarr:file')

Get the parameter schema writing multi-level datasets using the Levels format (\*.levels):

In [8]:
data_store.get_write_data_params_schema(writer_id='dataset:levels:file')

### Write multi-level dataset whose level 0 is a link

In [9]:
tile_size = 180
base_dataset_id = f"base_dataset_{tile_size}.zarr"
ml_dataset_id = f"ml_dataset_{tile_size}.levels" 

In [10]:
dataset = new_test_cube()

Explicitely define the spatial chunks (= tiles):

In [11]:
dataset = dataset.chunk(dict(lon=tile_size, lat=tile_size))

Write the base dataset that we'll turn into a multi-level dataset:

In [12]:
data_store.write_data(dataset, data_id=base_dataset_id, replace=True)

'base_dataset_180.zarr'

Write the 1+ levels of a multi-level dataset, level 0 remains the original dataset. A link to `base_dataset_id` is created instead: 

In [13]:
data_store.write_data(dataset, ml_dataset_id, replace=True, 
                      base_dataset_id=base_dataset_id)

'ml_dataset_180.levels'

In [14]:
list(data_store.get_data_ids())

['base_dataset_180.zarr', 'ml_dataset_180.levels']

Open the new multi-level dataset:

In [15]:
ml_dataset = data_store.open_data(ml_dataset_id)
ml_dataset

Inspect the new multi-level dataset. Check spatial chunking or `tile_size`:

In [16]:
ml_dataset.grid_mapping

class: **Coords1DGridMapping**
* is_regular: True
* is_j_axis_up: True
* is_lon_360: False
* crs: epsg:4326
* xy_res: (0.1, 0.1)
* xy_bbox: (-180, -90, 180, 90)
* ij_bbox: (0, 0, 3600, 1800)
* xy_dim_names: ('lon', 'lat')
* xy_var_names: ('lon', 'lat')
* size: (3600, 1800)
* tile_size: (180, 180)

In [17]:
ml_dataset.num_levels

5

In [18]:
for level in range(ml_dataset.num_levels):
    dataset_i = ml_dataset.get_dataset(level)
    display(dataset_i)

<xarray.Dataset>
Dimensions:    (time: 5, lat: 1800, lon: 3600, bnds: 2)
Coordinates:
  * lat        (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.8 179.9
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(5, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    A          (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
    B          (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 900, lon: 1800)
Coordinates:
  * lat      (lat) float64 -89.95 -89.75 -89.55 -89.35 ... 89.45 89.65 89.85
  * lon      (lon) float64 -179.9 -179.8 -179.5 -179.3 ... 179.4 179.6 179.8
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 450, lon: 900)
Coordinates:
  * lat      (lat) float64 -89.95 -89.55 -89.15 -88.75 ... 88.85 89.25 89.65
  * lon      (lon) float64 -179.9 -179.5 -179.1 -178.8 ... 178.8 179.2 179.6
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 225, lon: 450)
Coordinates:
  * lat      (lat) float64 -89.95 -89.15 -88.35 -87.55 ... 87.65 88.45 89.25
  * lon      (lon) float64 -179.9 -179.1 -178.3 -177.5 ... 177.6 178.4 179.2
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(5, 180, 180), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 113, lon: 225)
Coordinates:
  * lat      (lat) float64 -89.95 -88.35 -86.75 -85.15 ... 86.05 87.65 89.25
  * lon      (lon) float64 -179.9 -178.3 -176.8 -175.1 ... 175.2 176.8 178.4
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(5, 113, 180), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(5, 113, 180), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

### Write multi-level dataset with specified tile size

In [19]:
tile_size = 450
base_dataset_id = f"base_dataset_{tile_size}.zarr"
ml_dataset_id = f"ml_dataset_{tile_size}.levels" 

In [20]:
dataset = new_test_cube()

Write base dataset. No chunking specified, so we use the Zarr package's default chunking here.

In [21]:
data_store.write_data(dataset, data_id=base_dataset_id, replace=True)
dataset = data_store.open_data(base_dataset_id)
dataset

<xarray.Dataset>
Dimensions:    (time: 5, lat: 1800, lon: 3600, bnds: 2)
Coordinates:
  * lat        (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(1800, 2), meta=np.ndarray>
  * lon        (lon) float64 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.8 179.9
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(5, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    A          (time, lat, lon) int32 dask.array<chunksize=(1, 450, 900), meta=np.ndarray>
    B          (time, lat, lon) int32 dask.array<chunksize=(1, 450, 900), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

Write multi-level dataset and force spatial chunking to follow `tile_size`: 

In [22]:
data_store.write_data(dataset, data_id=ml_dataset_id, replace=True,  
                      tile_size=tile_size)

'ml_dataset_450.levels'

Open the new multi-level dataset:

In [23]:
ml_dataset = data_store.open_data(ml_dataset_id)
ml_dataset

Inspect the new multi-level dataset. Check spatial chunking or `tile_size`:

In [24]:
ml_dataset.grid_mapping

class: **Coords1DGridMapping**
* is_regular: True
* is_j_axis_up: True
* is_lon_360: False
* crs: epsg:4326
* xy_res: (0.1, 0.1)
* xy_bbox: (-180, -90, 180, 90)
* ij_bbox: (0, 0, 3600, 1800)
* xy_dim_names: ('lon', 'lat')
* xy_var_names: ('lon', 'lat')
* size: (3600, 1800)
* tile_size: (450, 450)

Number of levels decreased from 5 to 3:

In [25]:
ml_dataset.num_levels

3

In [26]:
for level in range(ml_dataset.num_levels):
    dataset_i = ml_dataset.get_dataset(level)
    display(dataset_i)

<xarray.Dataset>
Dimensions:    (time: 5, lat: 1800, lon: 3600, bnds: 2)
Coordinates:
  * lat        (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(1800, 2), meta=np.ndarray>
  * lon        (lon) float64 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.8 179.9
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * time       (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(5, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    A          (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    B          (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 900, lon: 1800)
Coordinates:
  * lat      (lat) float64 -89.95 -89.75 -89.55 -89.35 ... 89.45 89.65 89.85
  * lon      (lon) float64 -179.9 -179.8 -179.5 -179.3 ... 179.4 179.6 179.8
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube

<xarray.Dataset>
Dimensions:  (time: 5, lat: 450, lon: 900)
Coordinates:
  * lat      (lat) float64 -89.95 -89.55 -89.15 -88.75 ... 88.85 89.25 89.65
  * lon      (lon) float64 -179.9 -179.5 -179.1 -178.8 ... 178.8 179.2 179.6
  * time     (time) datetime64[ns] 2010-01-01T12:00:00 ... 2010-01-05T12:00:00
Data variables:
    A        (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
    B        (time, lat, lon) int32 dask.array<chunksize=(1, 450, 450), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.7
    geospatial_lat_max:    90.0
    geospatial_lat_min:    -90.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_max:    180.0
    geospatial_lon_min:    -180.0
    geospatial_lon_units:  degrees_east
    time_coverage_end:     2010-01-06T00:00:00
    time_coverage_start:   2010-01-01T00:00:00
    title:                 Test Cube